In [ ]:
from src.hpe.mp.performance import read_estimations as read_mp_estimations
from src.hpe.yolo.performance import read_estimations as read_yolo_estimations

In [ ]:
mp_est = read_mp_estimations(name="estimations")
mp_bgr_est = read_mp_estimations(name="estimations_bgr")

yolo_est = read_yolo_estimations(name="estimations")
yolo_bgr_est = read_yolo_estimations(name="estimations_bgr")

In [ ]:
from src.hpe.common.performance import log_performance

log_performance(mp_est, "MediaPipe RGB")
log_performance(mp_bgr_est, "MediaPipe BGR")
log_performance(yolo_est, "YOLO RGB")
log_performance(yolo_bgr_est, "YOLO BGR")

In [ ]:
from typing import List, Tuple
from matplotlib import pyplot as plt
from pandas import DataFrame
from numpy import ones, average, nan, isnan
from numpy.ma import masked_array

from src.hpe.common.landmarks import MyLandmark

In [ ]:
def plot_distances_boxplot(
        ylim: Tuple[int, int] | None  = None,
        save_location: str = "",
        *estimations: List[Tuple[str, DataFrame]]):
    
    def get_name(estimation: Tuple[str, DataFrame]) -> str:
        return estimation[0]
    
    def get_values(estimation: Tuple[str, DataFrame]) -> List[float]:

        def filter(x: HpeEstimation) -> bool:
            return x.can_predict and \
                x.true_landmark is not None and not x.true_landmark.is_missing() and \
                not x.predicted_landmark.is_missing()
        
        values: List[HpeEstimation] = estimation[1].values.reshape((-1))
        return [value.get_distance() / (value.head_bone_link / 2) for value in values if filter(value)]
    
    metrics = list(map(get_values, estimations))
    names = list(map(get_name, estimations))
    
    figure = plt.figure()
    plt.title("Comparison of relative distances")

    plt.axhline(1, c='r', label="PCKh50 limit", linestyle='--', linewidth=1)

    plt.boxplot(metrics, tick_labels=names)
    if ylim is not None:
        figure.axes[0].set_ylim(ylim)

    if save_location != "":
        plt.savefig(save_location)

def plot_absolute_distances_boxplot(
        ylim: Tuple[int, int] | None  = None,
        save_location: str = "",
        *estimations: List[Tuple[str, DataFrame]]):
    
    def get_name(estimation: Tuple[str, DataFrame]) -> str:
        return estimation[0]
    
    def get_values(estimation: Tuple[str, DataFrame]) -> List[float]:

        def filter(x: HpeEstimation) -> bool:
            return x.can_predict and \
                x.true_landmark is not None and not x.true_landmark.is_missing() and \
                not x.predicted_landmark.is_missing()
        
        values: List[HpeEstimation] = estimation[1].values.reshape((-1))
        return [value.get_distance() for value in values if filter(value)]
    
    metrics = list(map(get_values, estimations))
    names = list(map(get_name, estimations))
    
    figure = plt.figure()
    plt.title("Comparison of absolute distances")

    plt.boxplot(metrics, tick_labels=names)
    if ylim is not None:
        figure.axes[0].set_ylim(ylim)

    if save_location != "":
        plt.savefig(save_location)

In [ ]:
from os.path import join

In [ ]:
plot_absolute_distances_boxplot(
    None,
    join("data", "hpe", "absolute_distance_boxplot.png"),
    ("MediaPipe RGB", mp_est),
    ("MediaPipe BGR", mp_bgr_est),
    ("YOLO RGB", yolo_est),
    ("YOLO BGR", yolo_bgr_est))

In [ ]:
plot_distances_boxplot(
    (0, 10),
    join("data", "hpe", "distance_boxplot_zoomed.png"),
    ("MediaPipe RGB", mp_est),
    ("MediaPipe BGR", mp_bgr_est),
    ("YOLO RGB", yolo_est),
    ("YOLO BGR", yolo_bgr_est))

In [ ]:
from src.hpe.common.landmarks import YoloLabels, MyLandmark

def count_landmarks(labels: YoloLabels) -> int:
    
    def count_landmark(landmark: MyLandmark) -> int:
        label = labels.get_keypoint(landmark)
        return 0 if label.is_missing() else 1
    
    return sum(list(map(count_landmark, MyLandmark)))

In [ ]:
from os import listdir
from os.path import join

from src.hpe.common.landmarks import build_yolo_labels, get_most_central

root_label_dir = "data/hpe/img/test/labels"
totals = 0
    
for label_name in listdir(root_label_dir):
    label_path = join(root_label_dir, label_name)
    labels = build_yolo_labels(label_path)
    
    if len(labels) == 0:
        continue
    elif len(labels) == 1:
        totals += count_landmarks(labels[0])
    else:
        totals += count_landmarks(get_most_central(labels))



In [ ]:
totals / 41

In [ ]:
from src.hpe_dnn.kfold import HpeDnnFoldCrossValidation
from src.sota.kfold import SOTAFoldCrossValidation
from src.common.plot import box_plot_accuracies

box_plot_accuracies(
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-mp-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-mp-unbalanced-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-mp-dr0.3-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-yolo-kf"),
    # HpeDnnFoldCrossValidation.evaluation_instance("arch2-kf"), 
    # HpeDnnFoldCrossValidation.evaluation_instance("arch3-kf"), 
    # HpeDnnFoldCrossValidation.evaluation_instance("arch4-kf"), 
    # HpeDnnFoldCrossValidation.evaluation_instance("arch5-kf"),
    SOTAFoldCrossValidation.evaluation_instance("yolo11n-kf", "yolov11n-cls"))